# Check your installation

This notebook is meant to check if your infrastructure is supporting the requirements of this course. Since you are currently running this notebook, we guess you got the file with `git` !

In [20]:
!git --version

git version 2.39.2


## Xeus-cling C++17 kernel

If you have the relevant kernel installed, you should be able to execute the following code cells.

In [21]:
#include <iostream>

In [22]:
{
  int i = -1 ;
  unsigned int j = 1 ;
  if ( i < j ) std::cout << "i is less than j" ;
  else std::cout << "i is greater than j" ;
}

i is greater than j

## GCC

The more we progress in the lessons, the more advanced C++ features we will use: mostly C++17, and C++20/23 for few demonstrations. Even in the notebooks, when we want to practice the recent C++20 features, or the few C++17 features that are not supported by the xeus-cling notebook kernel, we will generate a file on-the-fly and compile it directly with g++. This course was prepared with g++ 13.2.

In [53]:
!g++ --version

g++ (GCC) 13.2.0
Copyright (C) 2023 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



Note: you will find [here](https://en.cppreference.com/w/cpp/compiler_support) the table of which C++ feature is supported by which compiler version. You should be aware that there is often a distinction between language syntax and standard library : a compiler version which supports C++20 language features does not necessarily supports the complete C++20 standard library. Generally, the support for syntax is implemented first, and the library novelties comes progressively with later versions.

## Gnu time

For some of the exercices, you will need to have **GNU time** installed where you want to execute the scripts. Try the command `\time -f "%U" echo toto` to check if it works fine. The `\` is there to avoid that bash use its own internal time command. If it does not work properly, try `conda install -c conda-forge time` (for example). 

In [3]:
!\time -f "%U" echo 42

42
0.00


## TBB backend

You should have TBB behind g++, if you want to observe the benefit of `std::execution::par`.

In [54]:
%%file tmp.execution.cpp

#include <vector>
#include <algorithm>
#include <chrono>
#include <string_view>
#include <iostream>
#include <execution>

template< typename Fonction, typename... ArgTypes >
auto time( std::string_view title, Fonction f, ArgTypes... args )
 {
  using namespace std::chrono ;
  auto t1 {steady_clock::now()} ;
  f(args...) ;
  auto t2 {steady_clock::now()} ;
  auto dt {duration_cast<microseconds>(t2-t1).count()} ;
  std::cout<<"("<<title<<" time: "<<dt<<" us)"<<std::endl ;
 }

void seq() {
  std::vector<double> xs(100000), ys(100000) ;
  std::transform( xs.begin(), xs.end(), ys.begin(), []( double x ) {
      double y {1.0} ;
      for ( int d=0 ; d<1024 ; ++d )
       { y *= x ; }
    return y ;
  });
}

void par() {
  std::vector<double> xs(100000), ys(100000) ;
  std::transform( std::execution::par, xs.begin(), xs.end(), ys.begin(), []( double x ) {
      double y {1.0} ;
      for ( int d=0 ; d<1024 ; ++d )
       { y *= x ; }
    return y ;
  });
}

int main() {
  time("seq",seq) ;
  time("par",par) ;
}

Writing tmp.execution.cpp


In [55]:
!rm -f tmp.execution.exe && g++ -std=c++17 -O2 -ltbb tmp.execution.cpp -o tmp.execution.exe

In [56]:
!./tmp.execution.exe

(seq time: 76667 us)
(par time: 13456 us)


## Eigen

The very end of the formation requires Eigen.

In [50]:
%%file tmp.eigen.cpp

#include <cmath>
double pow2( double d ) { return d*d ; }
double pow4( double d ) { return std::pow(d,4) ; }

#include "Eigen/Dense"
using Eigen3d = Eigen::Matrix<double,3,1> ;

#include <iostream>
int main() {
  double m { 9.109e-31 } ;
  double c { 299792458 } ;
  Eigen3d speed { 0.75*c, 0., 0. } ; 
  Eigen3d p { m*speed } ;
  double e = sqrt(pow2(m)*pow4(c)+pow2(p.norm())*pow2(c)) ;  
  std::cout << e << std::endl ;
}

Overwriting tmp.eigen.cpp


In [51]:
!rm -f tmp.eigen.exe && g++ -I/usr/local/include/eigen3/ -std=c++17 -O2 -ltbb tmp.eigen.cpp -o tmp.eigen.exe

In [52]:
!./tmp.eigen.exe

1.02335e-13
